In [1]:
import pandas as pd # data processing: CSV file I/O
import sweetviz as sv
import scipy as sc
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
DIR = 'datasets/'
anime = pd.read_csv(DIR + "anime.csv")
rating = pd.read_csv(DIR + "rating.csv")

In [169]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [5]:
anime_report = sv.analyze([anime, 'Anime'])
anime_report.show_html('anime_report.html', layout='vertical', scale=1.0)

                                             |          | [  0%]   00:00 -> (? left)

Report anime_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [6]:
rating_report = sv.analyze([rating, 'Anime Rating'])
rating_report.show_html('rating_report.html', layout='vertical', scale=1.0)

                                             |          | [  0%]   00:00 -> (? left)

Report rating_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [7]:
anime_rating_comparison = sv.compare([anime, 'Anime'],[rating, 'Anime Rating'], "anime_id")
anime_rating_comparison.show_html('rating_report.html', layout='vertical', scale=1.0)

                                             |          | [  0%]   00:00 -> (? left)

Report rating_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [8]:
valid_rating = rating[rating.rating != -1]

In [9]:
valid_rating.head()

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10


In [10]:
rating_validity: dict = {'total_ratings': rating['rating'].count(),
                        'valid_ratings': valid_rating['rating'].count(),
                         'invalid_ratings': rating['rating'].count() - valid_rating['rating'].count()}
print(rating_validity)

{'total_ratings': 7813737, 'valid_ratings': 6337241, 'invalid_ratings': 1476496}


In [11]:
anime.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


In [12]:
anime_ = anime.loc[:, {'anime_id', 'name'}]
merged_data = pd.merge(anime_, valid_rating)
user_anime_pivot = merged_data.pivot_table(index = ['user_id'], columns = ['anime_id'], values = 'rating').fillna(0)

In [13]:
user_anime_pivot.shape

(69600, 9926)

In [14]:
user_anime_pivot.tail()

anime_id,1,5,6,7,8,15,16,17,18,19,...,34238,34239,34240,34252,34283,34324,34325,34349,34367,34475
user_id,,,,,,,,,,,,,,,,,,,,,
73512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73513,9.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73515,10.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
user_anime_matrix = sc.sparse.csr_matrix(user_anime_pivot.values)
model_knn = NearestNeighbors(metric='euclidean', algorithm='brute')
model_knn.fit(user_anime_matrix)

NearestNeighbors(algorithm='brute', metric='euclidean')

In [237]:
seen_anime = []
def get_similar_users_by_userid(num_neighbors, user_id):
    global seen_anime
    if user_id != 0:
        seen_anime = valid_rating[valid_rating.user_id == user_id]['anime_id'].tolist()
    user_rating_reshaped = user_anime_pivot.loc[user_id,:].values.reshape(1,-1)
    distances, indices = model_knn.kneighbors(user_rating_reshaped, n_neighbors = num_neighbors)
    distances, user_ids = distances.flatten(), [user_anime_pivot.index[i] for i in indices.flatten()]
    print(f'Users similar to {user_id}:\n\nSN.\tUser_ID\t\tDistance')
    for i in range(0, len(distances)):
        print(f'{i}\t{user_ids[i]},\t\t{distances.flatten()[i]}')
    return user_ids

In [254]:
def get_similar_users_by_rating(num_neighbors, user_rating: dict):
    global seen_anime
    seen_anime = user_rating.keys
    user_anime_pivot.loc[0] = user_rating
    user_anime_pivot.loc[0] = user_anime_pivot.loc[0].fillna(0)
    return get_similar_users_by_userid(num_neighbors, 0)

In [255]:
# r = {9:5, 124:10, 234: 2, 983: 4, 99:8, 1064: 8}
# similar_user_ids = get_similar_users_by_rating(10, r)
similar_user_ids = get_similar_users_by_userid(10, 9837)
print(similar_user_ids)



Users similar to 9837:

SN.	User_ID		Distance
0	9837,		0.0
1	2753,		91.66787877986486
2	70543,		91.67333309092672
3	61629,		92.30926280715278
4	17167,		92.38506372785592
5	9762,		92.4824307639024
6	33877,		92.62828941527529
7	1077,		92.66067126888301
8	46759,		92.72000862812729
9	72545,		92.7739187487518
[9837, 2753, 70543, 61629, 17167, 9762, 33877, 1077, 46759, 72545]


In [256]:
def get_movies_recommendation(num_recommendations, user_ids):
    ratings_per_similar_user = valid_rating[valid_rating.user_id.isin(similar_user_ids)]
    unique_anime = ratings_per_similar_user['anime_id'].unique().tolist()
    unseen_anime = [anime for anime in unique_anime_from_ratings if anime not in seen_anime]
    anime_df = anime.isin(unseen_anime)

In [265]:
ratings_per_similar_user = valid_rating[valid_rating.user_id.isin(similar_user_ids)]
unique_anime = ratings_per_similar_user['anime_id'].unique().tolist()
unseen_anime = [anime for anime in unique_anime_from_ratings if anime not in seen_anime]
anime_df = anime[anime.anime_id.isin(unseen_anime)]
sorted_anime_df = anime_df.sort_values(by=['rating'], ascending=False)

In [263]:
len(unseen_anime)

115

In [264]:
anime_df

,anime_id,name,genre,type,episodes,rating,members
10,4181,Clannad: After Story,"Drama, Fantasy, Romance, Slice of Life, Supern...",TV,24,9.06,456749
16,23273,Shigatsu wa Kimi no Uso,"Drama, Music, Romance, School, Shounen",TV,22,8.92,416397
21,44,Rurouni Kenshin: Meiji Kenkaku Romantan - Tsui...,"Action, Drama, Historical, Martial Arts, Roman...",OVA,4,8.83,129307
24,164,Mononoke Hime,"Action, Adventure, Fantasy",Movie,1,8.81,339556
25,7311,Suzumiya Haruhi no Shoushitsu,"Comedy, Mystery, Romance, School, Sci-Fi, Supe...",Movie,1,8.81,240297
...,...,...,...,...,...,...,...
2878,4983,Hells,"Adventure, Comedy, Horror",Movie,1,7.14,9255
2898,22687,Terra Formars,"Action, Drama, Horror, Sci-Fi, Seinen, Space",TV,13,7.14,95964
3123,11285,Black★Rock Shooter (TV),"Action, Drama, School, Slice of Life",TV,8,7.07,198991
3398,11837,Zetman,"Action, Drama, Horror, Romance, Sci-Fi, Seinen...",TV,13,7.01,65633


In [266]:
sorted_anime_df

,anime_id,name,genre,type,episodes,rating,members
10,4181,Clannad: After Story,"Drama, Fantasy, Romance, Slice of Life, Supern...",TV,24,9.06,456749
16,23273,Shigatsu wa Kimi no Uso,"Drama, Music, Romance, School, Shounen",TV,22,8.92,416397
21,44,Rurouni Kenshin: Meiji Kenkaku Romantan - Tsui...,"Action, Drama, Historical, Martial Arts, Roman...",OVA,4,8.83,129307
24,164,Mononoke Hime,"Action, Adventure, Fantasy",Movie,1,8.81,339556
25,7311,Suzumiya Haruhi no Shoushitsu,"Comedy, Mystery, Romance, School, Sci-Fi, Supe...",Movie,1,8.81,240297
...,...,...,...,...,...,...,...
2878,4983,Hells,"Adventure, Comedy, Horror",Movie,1,7.14,9255
2898,22687,Terra Formars,"Action, Drama, Horror, Sci-Fi, Seinen, Space",TV,13,7.14,95964
3123,11285,Black★Rock Shooter (TV),"Action, Drama, School, Slice of Life",TV,8,7.07,198991
3398,11837,Zetman,"Action, Drama, Horror, Romance, Sci-Fi, Seinen...",TV,13,7.01,65633
